### **Exception Handling — Custom Exceptions, Logging & Best Practices**

#### **We’ll cover:**
- Creating your own exceptions
- Exception hierarchy
- Using raise from (exception chaining)
- Logging exceptions properly
- Re-raising exceptions
- Real-world patterns

**Basic custom exception**

In [29]:
# Define a custom exception class for insufficient funds
class InsufficientFundsError(Exception):
    """This exception is raised when a withdrawal exceeds the available balance."""
    pass  # The pass statement is used because no extra logic is needed in this case

# Function to perform a withdrawal
def withdraw(balance, amount):
    # Check if the withdrawal amount is greater than the available balance
    if amount > balance:
        # Raise the custom InsufficientFundsError with a descriptive message
        raise InsufficientFundsError(f"Available={balance}, Requested={amount}")
    
    # If the withdrawal is valid, return the new balance after subtraction
    return balance - amount

# Try block to attempt the withdrawal
try:
    # Trying to withdraw 1500 from an account with a balance of 1000
    withdraw(1000, 1500)

# Catch the custom InsufficientFundsError if it's raised
except InsufficientFundsError as e:
    # Print the custom error message from the exception
    print("⚠️", e)


⚠️ Available=1000, Requested=1500


**Exception hierarchy for large projects**

In [34]:
# Base class for all bank-related errors
class BankError(Exception):
    """Base class for all bank-related errors."""
    pass  # This class will be inherited by more specific bank-related exceptions

# Custom exception for insufficient funds
class InsufficientFundsError(BankError):
    """Raised when a transfer exceeds the available balance."""
    pass  # Inherits from BankError, no extra functionality here

# Custom exception for invalid account numbers
class InvalidAccountError(BankError):
    """Raised when an invalid account is used in a transfer."""
    pass  # Inherits from BankError, no extra functionality here

# Function to perform a money transfer between accounts
def transfer(src, dest, amount):
    # Check if the source account is valid
    if src not in ["A001", "A002"]:
        # If the source account is invalid, raise InvalidAccountError
        raise InvalidAccountError(f"Invalid account: {src}")
    
    # Check if the transfer amount exceeds the allowed limit
    if amount > 5000:
        # If the transfer amount exceeds the limit, raise InsufficientFundsError
        raise InsufficientFundsError("Limit exceeded")
    
    # If both checks pass, simulate the transfer (actual transfer logic can go here)
    print(f"Transferred {amount} from {src} to {dest}")

# Try to transfer from an invalid account
try:
    transfer("A003", "A002", 1000)  # Invalid source account: "A003"
except BankError as e:  # Catch all exceptions derived from BankError
    print("💰 Bank error:", e)


💰 Bank error: Invalid account: A003


**Adding attributes (richer error info)**

In [35]:
# Custom exception for data validation errors
class DataValidationError(Exception):
    # Initialize the exception with the column, value, and an optional message
    def __init__(self, column, value, message="Invalid data"):
        self.column = column  # Store the column name where the error occurred
        self.value = value    # Store the invalid value that caused the error
        # Initialize the base class (Exception) with a custom error message
        super().__init__(f"{message}: {column}={value}")

# Raise the custom exception with column="age" and an invalid value of -5
try:
    raise DataValidationError("age", -5)
except DataValidationError as e:
    # Print the details of the exception
    print("Column:", e.column)  # Access the 'column' attribute of the exception
    print("Value:", e.value)    # Access the 'value' attribute of the exception

Column: age
Value: -5


**Logging exceptions (instead of print)**

In [38]:
import logging

# Configuring the logging module
logging.basicConfig(
    filename="app_errors.log",           # Log messages will be written to "app_errors.log"
    level=logging.ERROR,                 # Only ERROR level and higher will be captured
    format="%(asctime)s [%(levelname)s] %(message)s"  # Log format with timestamp, log level, and message
)

# Simulating an error (division by zero)
try:
    10 / 0  # This will raise a ZeroDivisionError
except Exception as e:
    # Log the exception with traceback (logging.exception does this automatically)
    logging.exception("Unhandled error occurred")

**Combining with finally**

In [40]:
import logging

# Configuring logging
logging.basicConfig(
    filename="app_errors.log",
    level=logging.ERROR,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

def process_file(path):
    f = None
    try:
        # Try to open the file for reading
        f = open(path, "r")
        data = f.read()
        print("Read success:", data[:10])  # Print the first 10 characters of the file
    except Exception as e:
        # Log the exception with traceback if an error occurs
        logging.exception("File processing failed")
        raise  # Re-raise the exception to propagate it upward
    finally:
        # Ensure that the file is closed even if an error occurred
        if f:
            f.close()

# Example usage (uncomment to test)
# process_file("example.txt")


**Graceful error propagation (real-world ETL pattern)**

In [42]:
import logging

# Setting up logging configuration
logging.basicConfig(
    filename="etl_errors.log",
    level=logging.ERROR,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

# Base class for ETL-related errors
class ETLError(Exception): pass

# Custom exceptions for each ETL step
class ExtractError(ETLError): pass
class TransformError(ETLError): pass
class LoadError(ETLError): pass

# Simulating the extraction phase with a FileNotFoundError
def extract():
    raise FileNotFoundError("source.csv missing")

# Simulating the transformation phase with a ValueError
def transform(data):
    raise ValueError("Bad column type")

# Simulating the loading phase (no error in this example)
def load(data):
    print("Uploading to DB...")

# ETL Pipeline Execution
try:
    try:
        # Extraction phase: Raises FileNotFoundError
        data = extract()
    except FileNotFoundError as e:
        # Catch and raise custom ExtractError with chaining
        raise ExtractError("Extraction step failed") from e

    try:
        # Transformation phase: Raises ValueError
        clean = transform(data)
    except ValueError as e:
        # Catch and raise custom TransformError with chaining
        raise TransformError("Transformation failed") from e

    # Loading phase (no errors in this example)
    load(clean)

# Catch all ETL-related errors
except ETLError as e:
    # Log the exception with traceback
    logging.exception("ETL pipeline failed: %s", e)
